In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customers/BankChurners.csv


In [2]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)
import catboost
from sklearn.metrics import accuracy_score, recall_score, precision_score , confusion_matrix , classification_report


Importing Data

In [3]:
train = pd.read_csv (r'/kaggle/input/credit-card-customers/BankChurners.csv')
print (train)

       CLIENTNUM     Attrition_Flag  Customer_Age Gender  Dependent_count  \
0      768805383  Existing Customer            45      M                3   
1      818770008  Existing Customer            49      F                5   
2      713982108  Existing Customer            51      M                3   
3      769911858  Existing Customer            40      F                4   
4      709106358  Existing Customer            40      M                3   
...          ...                ...           ...    ...              ...   
10122  772366833  Existing Customer            50      M                2   
10123  710638233  Attrited Customer            41      M                2   
10124  716506083  Attrited Customer            44      F                1   
10125  717406983  Attrited Customer            30      M                2   
10126  714337233  Attrited Customer            43      F                2   

      Education_Level Marital_Status Income_Category Card_Category  \
0    

Removing the stuff they asked to remove


In [4]:
y = train.Attrition_Flag
X = train.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Attrition_Flag'], axis=1)

Converting the classifier values to numerical values


In [5]:
X = pd.get_dummies(X, columns=[ 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category'])

In [6]:
X.isnull().sum()

CLIENTNUM                         0
Customer_Age                      0
Dependent_count                   0
Months_on_book                    0
Total_Relationship_Count          0
Months_Inactive_12_mon            0
Contacts_Count_12_mon             0
Credit_Limit                      0
Total_Revolving_Bal               0
Avg_Open_To_Buy                   0
Total_Amt_Chng_Q4_Q1              0
Total_Trans_Amt                   0
Total_Trans_Ct                    0
Total_Ct_Chng_Q4_Q1               0
Avg_Utilization_Ratio             0
Gender_F                          0
Gender_M                          0
Education_Level_College           0
Education_Level_Doctorate         0
Education_Level_Graduate          0
Education_Level_High School       0
Education_Level_Post-Graduate     0
Education_Level_Uneducated        0
Education_Level_Unknown           0
Marital_Status_Divorced           0
Marital_Status_Married            0
Marital_Status_Single             0
Marital_Status_Unknown      

In [7]:
Xnew = X.drop(columns=[ 'Gender_F', 'Education_Level_College', 
                                 'Marital_Status_Divorced', 'Income_Category_Unknown', 'Card_Category_Blue'])

In [8]:
Xnew.columns

Index(['CLIENTNUM', 'Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Gender_M', 'Education_Level_Doctorate', 'Education_Level_Graduate',
       'Education_Level_High School', 'Education_Level_Post-Graduate',
       'Education_Level_Uneducated', 'Education_Level_Unknown',
       'Marital_Status_Married', 'Marital_Status_Single',
       'Marital_Status_Unknown', 'Income_Category_$120K +',
       'Income_Category_$40K - $60K', 'Income_Category_$60K - $80K',
       'Income_Category_$80K - $120K', 'Income_Category_Less than $40K',
       'Card_Category_Gold', 'Card_Category_Platinum', 'Card_Category_Silver'],
      dtype='object')

Train Test Split

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

In [10]:
#cat_features = list(range(0, X.shape[1]))
#print(cat_features)

In [11]:
from catboost.utils import create_cd
feature_names = dict()
for column, name in enumerate(train):
    if column == 0:
        continue
    feature_names[column - 1] = name
    
create_cd(
    label=0, 
    cat_features=list(range(1, train.columns.shape[0])),
    feature_names=feature_names,
    output_path=os.path.join('train.cd')
)

Fit Model

In [12]:

from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1,
    # loss_function='CrossEntropy'
)
model.fit(
    X_train, y_train,
    verbose = True
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 0.5698715	total: 61.8ms	remaining: 247ms
1:	learn: 0.4831222	total: 68.2ms	remaining: 102ms
2:	learn: 0.4166906	total: 74.2ms	remaining: 49.5ms
3:	learn: 0.3668002	total: 80.1ms	remaining: 20ms
4:	learn: 0.3186321	total: 85.9ms	remaining: 0us
Model is fitted: True
Model params:
{'iterations': 5, 'learning_rate': 0.1}


Metrics

In [13]:

model = CatBoostClassifier()
model.fit(X_train, y_train)
# Predicting result for training set and validation set
predict_val_rf2 = model.predict(X_validation)


# Model Performance

print("Accuracy : ", accuracy_score(y_validation, predict_val_rf2) *  100)
print("Recall : ", recall_score(y_validation, predict_val_rf2, pos_label = 'Attrited Customer') *  100)
print("Precision : ", precision_score(y_validation, predict_val_rf2, pos_label = 'Attrited Customer') *  100)
print(confusion_matrix(y_validation, predict_val_rf2))
print(classification_report(y_validation, predict_val_rf2))

Learning rate set to 0.02517
0:	learn: 0.6595020	total: 6.08ms	remaining: 6.07s
1:	learn: 0.6211697	total: 12.2ms	remaining: 6.06s
2:	learn: 0.5922505	total: 18.2ms	remaining: 6.06s
3:	learn: 0.5661571	total: 25ms	remaining: 6.24s
4:	learn: 0.5417021	total: 31.1ms	remaining: 6.19s
5:	learn: 0.5195260	total: 37.3ms	remaining: 6.17s
6:	learn: 0.5002685	total: 43.5ms	remaining: 6.18s
7:	learn: 0.4810208	total: 50ms	remaining: 6.2s
8:	learn: 0.4598221	total: 56.2ms	remaining: 6.19s
9:	learn: 0.4404202	total: 62.3ms	remaining: 6.17s
10:	learn: 0.4193241	total: 68.6ms	remaining: 6.17s
11:	learn: 0.4046525	total: 74.8ms	remaining: 6.16s
12:	learn: 0.3909977	total: 81.2ms	remaining: 6.16s
13:	learn: 0.3771510	total: 87.3ms	remaining: 6.15s
14:	learn: 0.3651025	total: 93.5ms	remaining: 6.14s
15:	learn: 0.3503875	total: 99.7ms	remaining: 6.13s
16:	learn: 0.3385867	total: 106ms	remaining: 6.13s
17:	learn: 0.3289421	total: 112ms	remaining: 6.13s
18:	learn: 0.3212559	total: 119ms	remaining: 6.13s
1

In [14]:
tp = sum([1 if actual_y == predicted_y and actual_y == 'Attrited Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])
tn = sum([1 if actual_y == predicted_y and actual_y == 'Existing Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])
fp = sum([1 if actual_y != predicted_y and actual_y == 'Attrited Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])
fn = sum([1 if actual_y != predicted_y and actual_y == 'Existing Customer' else 0 for actual_y, predicted_y in zip(y_train,predict_val_rf2)])

In [15]:
print("Accuracy : ", accuracy_score(y_validation, predict_val_rf2) *  100)


Accuracy :  97.63079960513326
